In [2]:
from tensorflow.keras.applications import EfficientNetB2, EfficientNetB0
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l1_l2
from timeit import default_timer as timer
import numpy as np
import const
import os

In [14]:
# Définition des constantes

class TimingCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

time_callback = TimingCallback()


# Définition des chemins
# dataset_path = const.DATASET_REDUIT
dataset_path = const.DATASET_CLEAN_WO_BACKGROUND
# dataset_path = const.DATASET_CLEAN_PATH
# dataset_path = const.DATASET_CONTOUR
# dataset_path = const.DATASET_WO_BACKGROUND

diverted_images_train_path = os.path.join(dataset_path, "train")
diverted_images_valid_path = os.path.join(dataset_path, "valid")
diverted_images_test_path = os.path.join(dataset_path, "test")

# Définition de la taille du lot
batch_size = 16

# Définition des callbacks
reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)

early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')

# Création du générateurs d'images de test
test_generator = ImageDataGenerator().flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size)


In [15]:
# Construction du modèle
def build_model(num_classes, type_Efficient, kernel_regularizer=None, rate_Droput=0.3):
    if type_Efficient == "B0":
        base_model = EfficientNetB0(weights='imagenet', include_top=False)
    elif type_Efficient == "B2":
        base_model = EfficientNetB2(weights='imagenet', include_top=False)
    else:
        raise ValueError("type_Efficient devrait être B0 ou B2")
    for layer in base_model.layers[:-20]:  
        layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1280, activation='relu', kernel_regularizer=kernel_regularizer)(x) 
    x = Dropout(rate=rate_Droput)(x)
    x = Dense(640, activation='relu', kernel_regularizer=kernel_regularizer)(x)  
    x = Dropout(rate=rate_Droput)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
#modèle EfficientNet amélioré - meilleur modèle obtenu - val_acc 0.9561 - dataset images détourées
# Création des générateurs d'images avec augmentation des données
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size)
valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)

# Récupération du nombre de classes
num_classes = train_generator.num_classes
model = build_model(num_classes, type_Efficient="B2", rate_Droput=0.2)

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

# Entraînement du modèle
history_diverted = model.fit(train_generator,
                    epochs=15,  
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

# Évaluation du modèle
test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


In [ ]:
#modèle EfficientNet val_acc 0.91 - dataset images détourées

# Création des générateurs d'images
train_generator = ImageDataGenerator().flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size)
valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)

# Récupération du nombre de classes
num_classes = train_generator.num_classes

model = build_model(num_classes, type_Efficient="B0", rate_Droput=0.2)

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

# Entraînement du modèle
history_diverted = model.fit(train_generator,
                    epochs=15,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

# Évaluation du modèle
test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


In [ ]:
# Création des générateurs d'images avec augmentation des données
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size)

valid_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)

num_classes = len(train_generator.class_indices)
model = build_model(num_classes, type_Efficient="B2", kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))

# Compilation du modèle
model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

# Entraînement du modèle
history_diverted = model.fit(train_generator,
                    epochs=10,  
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

# Évaluation du modèle
test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


In [ ]:
# Création des générateurs d'images avec augmentation des données
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size)

valid_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)

num_classes = len(train_generator.class_indices)
model = build_model(num_classes, type_Efficient="B0", kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))
# Compilation du modèle
model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

# Entraînement du modèle
history_diverted = model.fit(train_generator,
                    epochs=10,  
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

# Évaluation du modèle
test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])

In [ ]:
# Création des générateurs d'images avec augmentation des données
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size)

valid_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)

num_classes = len(train_generator.class_indices)
model = build_model(num_classes, type_Efficient="B0")

# Compilation du modèle
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

# Entraînement du modèle
history_diverted = model.fit(train_generator,
                    epochs=20,  
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

# Évaluation du modèle
test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])

In [ ]:
# Création des générateurs d'images avec augmentation des données
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size)

valid_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)

num_classes = len(train_generator.class_indices)
model = build_model(num_classes, type_Efficient="B0")

# Compilation du modèle
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

# Entraînement du modèle
history_diverted = model.fit(train_generator,
                    epochs=20,  
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

# Évaluation du modèle
test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])